In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import tensorflow as tf
import keras_tuner as kt
from tensorflow import keras
from pathlib import Path

print(tf.__version__)
#veri setinin indirilip dosya yollarının kaydedilmesi

movielens_url = "http://files.grouplens.org/datasets/movielens/ml-latest-small.zip"

downloaded_path = tf.keras.utils.get_file(movielens_url.rsplit('/').pop(), movielens_url, extract=True)

dataset_dir = Path(downloaded_path).parent / Path(downloaded_path).stem
print(dataset_dir)

ratings_file = dataset_dir / "ratings.csv"
ratings = pd.read_csv(ratings_file)
movies_file = dataset_dir / "movies.csv"
movies = pd.read_csv(movies_file)
tags_file = dataset_dir / "tags.csv"
tags = pd.read_csv(tags_file)


2022-12-30 19:24:07.327766: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-30 19:24:07.483856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-30 19:24:07.483876: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-30 19:24:08.960203: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

2.11.0
/home/feindrial/.keras/datasets/ml-latest-small


In [2]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
5,1,70,3.0,964982400
6,1,101,5.0,964980868
7,1,110,4.0,964982176
8,1,151,5.0,964984041
9,1,157,5.0,964984100


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [5]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [6]:
#TODO sklearn.MultiLabelBinarizer kullanılarak film türleri aynı şekilde haritalanabilir

unique_genres = pd.unique(movies["genres"].str.split("|", expand=True).values.ravel())
movies[unique_genres] = 0

movies = movies.astype({x: np.int8 for x in unique_genres})

for index, row in enumerate(movies["genres"].str.split("|")):
    movies.loc[index, row] = 1

movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             9742 non-null   int64 
 1   title               9742 non-null   object
 2   genres              9742 non-null   object
 3   Adventure           9742 non-null   int8  
 4   Animation           9742 non-null   int8  
 5   Children            9742 non-null   int8  
 6   Comedy              9742 non-null   int8  
 7   Fantasy             9742 non-null   int8  
 8   None                9742 non-null   int8  
 9   Romance             9742 non-null   int8  
 10  Drama               9742 non-null   int8  
 11  Action              9742 non-null   int8  
 12  Crime               9742 non-null   int8  
 13  Thriller            9742 non-null   int8  
 14  Horror              9742 non-null   int8  
 15  Mystery             9742 non-null   int8  
 16  Sci-Fi              9742

In [7]:
movies = movies.drop(["genres", "title"], axis=1)
movies.head(10)

,movieId,Adventure,Animation,Children,Comedy,Fantasy,None,Romance,Drama,Action,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,1,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,6,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,8,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,9,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9,10,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ratings = ratings.join(movies.set_index("movieId"), on="movieId", how="inner")

In [9]:
ratings = ratings.drop("timestamp", axis=1)
ratings.head(10)

,userId,movieId,rating,Adventure,Animation,Children,Comedy,Fantasy,None,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,1,4.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
516,5,1,4.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
874,7,1,4.5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1434,15,1,2.5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1667,17,1,4.5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1772,18,1,3.5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2274,19,1,4.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3219,21,1,3.5,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4059,27,1,3.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4879,31,1,5.0,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100828
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   userId              100836 non-null  int64  
 1   movieId             100836 non-null  int64  
 2   rating              100836 non-null  float64
 3   Adventure           100836 non-null  int8   
 4   Animation           100836 non-null  int8   
 5   Children            100836 non-null  int8   
 6   Comedy              100836 non-null  int8   
 7   Fantasy             100836 non-null  int8   
 8   None                100836 non-null  int8   
 9   Romance             100836 non-null  int8   
 10  Drama               100836 non-null  int8   
 11  Action              100836 non-null  int8   
 12  Crime               100836 non-null  int8   
 13  Thriller            100836 non-null  int8   
 14  Horror              100836 non-null  int8   
 15  Mystery             100836 non-nul

In [11]:
num_unique_users = len(ratings["userId"].unique())
num_unique_movies = len(ratings["movieId"].unique())

min_rating = ratings["rating"].min()
max_rating = ratings["rating"].max()

x_train = ratings.drop("rating", axis=1).values
y_labels = ratings["rating"]

#film değerlendirmeleri ilgili puana göre katergorize edildi
y_train = keras.utils.to_categorical(sk.preprocessing.LabelEncoder().fit_transform(y_labels))

In [12]:
x_train[:3]

array([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0],
       [5, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0],
       [7, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]])

In [13]:
y_train[:3]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)

In [14]:
#TODO sınıf farklı veri setlerine göre kendini uyumlu hale getiremiyor, daha dinamik bir mimari uygulanması gerekli

class BondNet(keras.Model):
    def __init__(self, features: np.array, labels: np.array, num_unique_users, num_unique_movies, min_rating, max_rating, rating_step=0.5, embedding_size=32, **kwds):
        super(BondNet, self).__init__(**kwds)

        ##TODO veriyi ayırma statik olarak belirlenmemeli, train fonksiyonunda parametre olarak alınmalı 
        self.data_split = int(0.8 * len(features))

        self.features = features
        self.labels = labels
        self.embedding_size = embedding_size

        #EMBEDDING API
        
        self.user_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 0]), np.arange(num_unique_users)),
            default_value=-1
        )
        self.movie_table = tf.lookup.StaticHashTable(
            tf.lookup.KeyValueTensorInitializer(np.unique(features[:, 1]), np.arange(num_unique_movies)),
            default_value=-1
        )

        self.embedding_input = keras.layers.InputLayer(
            input_shape=(2,),
            dtype=np.int64
        )

        self.user_embedding = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.L2(l2=1e-3)
        )
        self.user_bias = keras.layers.Embedding(
            input_dim=num_unique_users,
            output_dim=1
        )

        self.movie_embedding = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=embedding_size,
            embeddings_initializer="he_normal",  
            embeddings_regularizer=keras.regularizers.L2(l2=1e-3)
        )
        self.movie_bias = keras.layers.Embedding(
            input_dim=num_unique_movies,
            output_dim=1
        )
        
        #VISIBLE FEATURES API

        #film türleri
        self.visible_input = keras.layers.InputLayer(
            input_shape=(len(self.features[0, 2:]),),
            dtype=np.int8
        )
        self.dense1 = keras.layers.Dense(
            1024,
            activation="relu"
        )
        self.dense2 = keras.layers.Dense(
            512,
            activation="relu"
        )
        
        self.visible_output = keras.layers.Dense(
            1,
            activation="sigmoid"
        )

        self.output_labels = keras.layers.Dense(
            int(((max_rating - min_rating) / 0.5) + 1),
            activation="softmax"
        )

    def call(self, inputs):
        user_index = self.user_table.lookup(inputs[:, 0])
        movie_index = self.movie_table.lookup(inputs[:, 1])

        user_vector = self.user_embedding(user_index)
        user_bias = self.user_bias(user_index)

        movie_vector = self.movie_embedding(movie_index)
        movie_bias = self.movie_bias(movie_index)

        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        x = dot_user_movie + user_bias + movie_bias

        genres_vector = self.dense2(self.dense1(self.visible_input(inputs[:, 2:])))
        genres_out = self.visible_output(genres_vector)             

        out = tf.concat([x, genres_out], axis=1)

        return self.output_labels(out)

    def get_train_dataset(self):
        x_train, y_train = (
            self.features[:self.data_split],
            self.labels[:self.data_split]
        )

        train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)
        return train_dataset

    def get_validation_dataset(self):
        x_val, y_val = (
            self.features[self.data_split:],
            self.labels[self.data_split:]
        )

        val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
        val_dataset = val_dataset.shuffle(buffer_size=1024).batch(64)
        return val_dataset

    #TODO model fonksiyonlarının kendi implementasyon biçimleri ile çağrılması gerek
    def cmpl():
        pass
    def trn():
        pass

In [42]:
bondnet = BondNet(
    x_train,
    y_train,
    num_unique_users,
    num_unique_movies,
    min_rating,
    max_rating,
    embedding_size=16
)

bondnet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-1),
    loss="MeanSquaredLogarithmicError",
    metrics=[
        tf.keras.metrics.MeanSquaredError(),
        tf.keras.metrics.RootMeanSquaredError(),
        "accuracy"
    ]
)

In [43]:
td = bondnet.get_train_dataset()
vd = bondnet.get_validation_dataset()

history = bondnet.fit(
    td,
    epochs=1,
    validation_data=vd
)
history.history

1261/1261 [==============================] - 11s 8ms/step - loss: 0.0422 - mean_squared_error: 0.0805 - root_mean_squared_error: 0.2837 - accuracy: 0.3106 - val_loss: 0.0419 - val_mean_squared_error: 0.0845 - val_root_mean_squared_error: 0.2906 - val_accuracy: 0.2522


{'loss': [0.04224833846092224],
 'mean_squared_error': [0.08050508797168732],
 'root_mean_squared_error': [0.28373420238494873],
 'accuracy': [0.310606449842453],
 'val_loss': [0.041948869824409485],
 'val_mean_squared_error': [0.08446888625621796],
 'val_root_mean_squared_error': [0.29063525795936584],
 'val_accuracy': [0.25223127007484436]}

In [41]:
rnd = np.random.randint(0, len(x_train))

print(bondnet.predict(x_train[rnd - 1:rnd]))
print(y_train[rnd - 1:rnd])

1/1 [==============================] - 0s 19ms/step
[[0.02646451 0.06036795 0.03172698 0.09330577 0.05241523 0.21743299
  0.08514608 0.2915682  0.07459197 0.06698034]]
[[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


In [44]:
bondnet.summary()

Model: "bond_net_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 2)]               0         
                                                                 
 embedding_24 (Embedding)    multiple                  9760      
                                                                 
 embedding_25 (Embedding)    multiple                  610       
                                                                 
 embedding_26 (Embedding)    multiple                  155584    
                                                                 
 embedding_27 (Embedding)    multiple                  9724      
                                                                 
 input_14 (InputLayer)       [(None, 21)]              0         
                                                                 
 dense_24 (Dense)            multiple                  2